라이브러리들

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage,HumanMessage,SystemMessage
from langchain.prompts import PromptTemplate,ChatPromptTemplate,load_prompt,MessagesPlaceholder
from langchain.schema import BaseOutputParser
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotPromptTemplate,FewShotChatMessagePromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.globals import set_llm_cache,set_debug
from langchain.cache import InMemoryCache
from langchain.callbacks import get_openai_callback
from langchain.memory import ConversationBufferMemory,ConversationBufferWindowMemory,ConversationSummaryBufferMemory,ConversationSummaryMemory,ConversationKGMemory
from langchain.chains import LLMChain
from langchain.schema.runnable import RunnablePassthrough





자주 쓰는 변수들

In [2]:
CHAT = ChatOpenAI(
    temperature=0.5,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

#3.0

In [ ]:
a = CHAT.predict("좀 있다가 에슐리 퀸즈를 가는데, 먹을거 추천 좀 해줘봐")
# 예제에서 특징은 chat이라는 변수에 어떠한 모델을 사용하든, predict라는 메소드 사용 가능 / 답변은 매번 바뀜

a

#3.1

In [ ]:
message = [
    SystemMessage(
        content="넌 이제부터 매우 awsome하고 coooool한 음식의 specialist야. 그 어떠한 음식에 대해 물어보면 재료와 레시피에 대해 말할 수 있어."),
    AIMessage(
        content="안녕하세요. 저는 맛있는 녀석입니다."),
    HumanMessage(
        content="일식 덮밥집에서 파는 부타동에 대해 설명해줘.")
]

CHAT.predict_messages(message)

#3.2

In [ ]:
prom_template = PromptTemplate.from_template("{place}을(를) 가는데, 추천 좀 해줘봐")
#PromptTemplate는 단순 메세지를 프롬프트로 사용 할 때 유용함

prom_template = prom_template.format(place = "맘스터치")

CHAT.predict(prom_template)

In [ ]:
chat_template = ChatPromptTemplate.from_messages([
    ("system", "넌 이제부터 매우 awsome하고 coooool한 {job}의 specialist야. 그 어떠한 질문이든 대답 할 수 있어."),
    ("ai", "안녕하세요. 저는 {name}입니다."),
    ("human", "{place}을(를) 가는데, 추천 좀 해줘."),
])
#ChatPromptTemplate는 조금 더 정교하게 설정된 프롬프트를 탬플릿을 통해 사용 할 때 유용함


chat_template = chat_template.format(job="음식",name="맛있는 녀석",place="맘스터치")

CHAT.predict(chat_template)

#3.3

In [ ]:
#BaseOutputParser를 사용하여 결과 출력을 parse하기

class CommaOutParser(BaseOutputParser):
    def parse(self,text): # 필수 메소드
        items = text.strip().split(".")
        return(list(map(str.strip,items)))
    
parser = CommaOutParser()
parser.parse("  안녕하세요     .   유령개입니다 .    잘부탁드리겠습니다.")

#3분 24초

In [ ]:
chat_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 질문이 들어오면, 최대 {n}개만큼의 리스트로 대답하는 리스트 대답 기계입니다. 리스트가 아닌걸론 대답하지 마세요. 그리고, 각 리스트들은 ,(콤마)로 구분되야해요."),
    ("human", "{question}"),
])

chat_template = chat_template.format(
    n = 10,
    question = "아시아에 위치한 나라를 알려주세요.  "
)

rs = CHAT.predict(chat_template)
parser.parse(rs)

#-------------------------

In [ ]:
# 위의 두개와 달리, 더 간단하게 할 수 있는 버전이 있다. 여기선 오직 CHAT이라는 chatmodel 변수와, template. parser 세 개만이 존재한다.

class CommaOutParser(BaseOutputParser):
    def parse(self,text): # 필수 메소드
        items = text.strip().split(".")
        return(list(map(str.strip,items)))

chat_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 질문이 들어오면, 최대 {n}개만큼의 리스트로 대답하는 리스트 대답 기계입니다. 리스트가 아닌걸론 대답하지 마세요. 그리고, 각 리스트들은 ,(콤마)로 구분되야해요."),
    ("human", "{question}"),
])


chain = chat_template | CHAT | CommaOutParser()
chain.invoke({
    "n":3,
    "question":"유럽의 나라"
})

#3.4

In [ ]:
chef_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 세계적인 슈퍼 클래스의 요리사입니다. 어떠한 요리를 요구하든 그 요리의 재료와 레시피에 대해 알려줄 수 있습니다."),
    ("human", "저는 {food} 요리를 만들고 싶습니다. 들어가는 재료와 레시피에 대해 알려주실 수 있을까요?"),
])
veg_chef_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 어떠한 요리의 레시피가 들어와도 채식으로 변형시킬 수 있는 세계 대표의 채식 요리사입니다. 레시피가 들어오면, 해당 메뉴의 재료를 채식에 맞게 변형시켜야합니다."),
    ("human", "{recipe}"),
])

chef_chain = chef_template | CHAT
veg_chef_chain = veg_chef_template | CHAT
fianal_chain = {"recipe":chef_chain} | veg_chef_chain # chef_template | CHAT | veg_chef_template | CHAT << 이것과 같은 형태임.
# 여러개의 체인을 위와 같은 방식으로 엮을 수 있음. 이건 머신러닝과 엇비슷한 형태라고 볼 수 있음

fianal_chain.invoke({
    "food": "프랑스"
})


#4.0

In [ ]:
# fewShort는 모델에게 예제를 준다. > 더 나은 대답을 받기 위해 / 프롬프트의 system에서 ai에거 답변 요구사항을 올리는것보단, 예제로 출력 형식을 잡아주는게 더 효율적임

# 샘플들 > 이제 ai가 이러한 형태로 답변을 하게 됨

few_short_example = [
    {
        "question":"터키의 수도에 대해 알려줘.",
        "answer":"""
        터키의 수도는 '앙카라'입니다. 면적은 면적은 약 2516㎢이며, 인구는 2010년 기준 4,431,719명입니다.
        '앙카라'라는 지명은 '닻'을 의미하는 그리스어 앙퀴라에서 유래된 것으로 추측됩니다.
        '앙카라'에는 앙카라 성채, 아나톨리아 문명박물관, 악칼레, 등과 같은 관광지가 존재합니다.
        """
    },
    {
        "question":"프랑스의 수도에 대해 알려줘.",
        "answer":"""
        프랑스의 수도는 '파리'입니다. 면적은 면적은 105.4㎢이며, 인구는 2023년 기준 13,024,518명입니다.
        '파리'라는 지명은 고대에 이 지역에 살았던 갈리아의 일족 파리시(Parisii)에서 유래되었습니다.
        '파리'에는 에펠탑, 루브루 박물관 등과 같은 관광지가 존재합니다.
        """
    },
    {
        "question":"한국의 수도에 대해 알려줘.",
        "answer":"""
        한국의 수도는 '서울'입니다. 면적은 면적은 약 605.2㎢이며, 인구는 2024년 기준 9,360,400명입니다.
        '서울'라는 지명은 한국어의 순우리말로서 '한 나라의 수도(首都)', 곧 '국도(國都)'를 가리키는 일반명사입니다.
        '서울'에는 광하문, 국립중앙박물관, 경복궁, 등과 같은 관광지가 존재합니다.
        """
    },
]

In [ ]:
# FewShotPromptTemplate는 PromptTemplate 기반으로 예제를 사용함

example_template ="""
    Human:{question}
    AI:{answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=few_short_example,
    suffix="Human: {country}의 수도에 대해 알려줘.",
    input_variables=["country"]
)

chain = prompt | CHAT
chain.invoke({"country":"아시리아"})

#4.2

In [ ]:
few_short_chat_example = [
    {
        "country":"터키",
        "answer":"""
        터키의 수도는 '앙카라'입니다. 면적은 면적은 약 2516㎢이며, 인구는 2010년 기준 4,431,719명입니다.
        '앙카라'라는 지명은 '닻'을 의미하는 그리스어 앙퀴라에서 유래된 것으로 추측됩니다.
        '앙카라'에는 앙카라 성채, 아나톨리아 문명박물관, 악칼레, 등과 같은 관광지가 존재합니다.
        """
    },
    {
        "country":"프랑스",
        "answer":"""
        프랑스의 수도는 '파리'입니다. 면적은 면적은 105.4㎢이며, 인구는 2023년 기준 13,024,518명입니다.
        '파리'라는 지명은 고대에 이 지역에 살았던 갈리아의 일족 파리시(Parisii)에서 유래되었습니다.
        '파리'에는 에펠탑, 루브루 박물관 등과 같은 관광지가 존재합니다.
        """
    },
    {
        "country":"한국",
        "answer":"""
        한국의 수도는 '서울'입니다. 면적은 면적은 약 605.2㎢이며, 인구는 2024년 기준 9,360,400명입니다.
        '서울'라는 지명은 한국어의 순우리말로서 '한 나라의 수도(首都)', 곧 '국도(國都)'를 가리키는 일반명사입니다.
        '서울'에는 광하문, 국립중앙박물관, 경복궁, 등과 같은 관광지가 존재합니다.
        """
    },
]

In [ ]:
# FewShotChatMessagePromptTemplate은 ChatPromptTemplate을 기반으로하여 예제를 사용함

example_prompt = ChatPromptTemplate.from_messages([
        ("user","{country}의 수도에 대해 알려줘."),
        ("ai","{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=few_short_chat_example,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system","당신은 세계적인 지리 및 역사학자입니다. 특정 국가에 대해 물어보면 그게 얼마나 오래된 국가여도 대답해줄 수 있습니다."),
    example_prompt,
    ("human","{country}의 수도에 대해 알려줘.")
])

chain = final_prompt | CHAT
chain.invoke({"country": "몽골"})

#4.3

In [ ]:
# 예시를 랜덤으로 선택해주는 ExampleSelector. BaseExampleSelector를 상속받음.
class ExampleSelector(BaseExampleSelector):

    def __init__(self,examples) -> None:
        self.examples = examples    # 시작할때, 샘플을 받음.

    def select_examples(self, input_variables): #파라미터로 받은 샘플을 고름
        from random import choice
        return [choice(self.examples)]
    
    def add_example(self, example):
        self.examples.append(example)

In [ ]:
example_prompt = PromptTemplate.from_template(example_template)

example_selector = LengthBasedExampleSelector(
    examples=few_short_example,
    example_prompt=example_prompt,
    max_length=50
)

base_example_selector = ExampleSelector(
    examples=few_short_example
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    suffix="Human: {country}의 수도에 대해 알려줘.",
    input_variables=["country"],
    example_selector=base_example_selector
)

prompt.format(country="미국")


#4.4

In [ ]:
# 인코딩 에러 발생해서 아래와 같은 코드로 구성

import json
from langchain.prompts import PromptTemplate

# JSON 파일을 UTF-8로 읽기
with open("./promt.json", "r", encoding="utf-8") as f:
    config = json.load(f)

# JSON 파일의 내용을 기반으로 PromptTemplate 생성
j_promt = PromptTemplate(
    input_variables=config["input_variables"],
    template=config["template"]
)

# 프롬프트 포맷 사용
formatted_output = j_promt.format(country="호주")
print(formatted_output)
CHAT.predict(formatted_output)

In [ ]:
# 인코딩 에러 발생해서 아래와 같은 코드로 구성

import yaml
from langchain.prompts import PromptTemplate

# YAML 파일을 UTF-8로 읽기
with open("./promt.yaml", "r", encoding="utf-8") as f:
    config = yaml.safe_load(f)

# YAML 파일의 내용을 기반으로 PromptTemplate 생성
y_promt = PromptTemplate(
    input_variables=config["input_variables"],
    template=config["template"]
)

# 프롬프트 포맷 사용
formatted_output = y_promt.format(country="호주")
print(formatted_output)
CHAT.predict(formatted_output)


In [ ]:
# 다양한 프롬프트를 하나로 묶는 방법으로 PipelinePromptTemplate이 있음
#탬플릿1
intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)
#탬플릿2
example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)
#탬플릿3
start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)
#3개의 탬플릿을 합치기 위한 틀
final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)
#prompts와 full_prompt을 통해 3개의 탬플릿을 묶음
prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]


full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)

# chain 제작
chain = full_prompt | CHAT

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

#4.5

In [13]:
# set_llm_cache(InMemoryCache()) #캐시 메모리에 대화를 저장하는 코드.
# set_debug(True) #디버그 모드 활성화. 로그를 더 상세하게 볼 수 있음

# CHAT.predict("이탈리아 화덕 피자 만드는 방법")

In [14]:
CHAT.predict("이탈리아 화덕 피자 만드는 방법")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: 이탈리아 화덕 피자 만드는 방법"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "이탈리아 화덕 피자를 만드는 방법은 다음과 같습니다:\n\n1. 피자 반죽을 만듭니다: 밀가루, 물, 소금, 이스트를 섞어 부드러운 반죽을 만들어줍니다. 이를 냉장고에서 발효시켜줍니다.\n\n2. 토마토 소스를 만듭니다: 토마토 퓨레, 올리브 오일, 소금, 후추, 마늘을 섞어 소스를 만들어줍니다.\n\n3. 토핑을 준비합니다: 모차렐라 치즈, 페퍼로니, 버섯, 양파, 파프리카 등을 썰어 준비해둡니다.\n\n4. 화덕을 예열합니다: 이탈리아 화덕은 높은 열을 유지할 수 있는 화덕이기 때문에 높은 열을 유지할 수 있도록 예열해줍니다.\n\n5. 반죽을 펴서 피자를 만듭니다: 반죽을 펴서 동그랗게 모양을 만들고, 토마토 소스를 발라줍니다. 그리고 토핑을 얹어줍니다.\n\n6. 화덕에 넣고 굽습니다: 화덕 속에 피자를 넣고 약 1~2분간 굽습니다. 이때 피자를 자주 확인하여 너무 타지 않도록 주의해야 합니다.\n\n7. 피자를 꺼내고 마무리합니다: 피자를 꺼내서 올리브 오일이나 파슬리를 뿌려 마무리해줍니다.\n\n이렇게 하면 이탈리아 화덕 피자를 만들 수 있습니다. 맛있게 즐겨보세요!",
        "generation_info": {
          "finish_reason": "stop"
        },
        "type": "ChatGenerationChunk",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            

'이탈리아 화덕 피자를 만드는 방법은 다음과 같습니다:\n\n1. 피자 반죽을 만듭니다: 밀가루, 물, 소금, 이스트를 섞어 부드러운 반죽을 만들어줍니다. 이를 냉장고에서 발효시켜줍니다.\n\n2. 토마토 소스를 만듭니다: 토마토 퓨레, 올리브 오일, 소금, 후추, 마늘을 섞어 소스를 만들어줍니다.\n\n3. 토핑을 준비합니다: 모차렐라 치즈, 페퍼로니, 버섯, 양파, 파프리카 등을 썰어 준비해둡니다.\n\n4. 화덕을 예열합니다: 이탈리아 화덕은 높은 열을 유지할 수 있는 화덕이기 때문에 높은 열을 유지할 수 있도록 예열해줍니다.\n\n5. 반죽을 펴서 피자를 만듭니다: 반죽을 펴서 동그랗게 모양을 만들고, 토마토 소스를 발라줍니다. 그리고 토핑을 얹어줍니다.\n\n6. 화덕에 넣고 굽습니다: 화덕 속에 피자를 넣고 약 1~2분간 굽습니다. 이때 피자를 자주 확인하여 너무 타지 않도록 주의해야 합니다.\n\n7. 피자를 꺼내고 마무리합니다: 피자를 꺼내서 올리브 오일이나 파슬리를 뿌려 마무리해줍니다.\n\n이렇게 하면 이탈리아 화덕 피자를 만들 수 있습니다. 맛있게 즐겨보세요!'

#4.6

In [16]:
with get_openai_callback() as usage: #usage를 통해 모델에서 사용한 토큰 수 및 비용 등의 데이터를 볼 수 있음
    CHAT.predict("와인을 만드는 방법")
    print(usage)

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: 와인을 만드는 방법"
  ]
}
와인을 만드는 과정은 다음과 같습니다:

1. 수확: 와인을 만들기 위한 포도를 수확합니다. 수확 시기는 포도의 성숙도와 품종에 따라 다르며, 보통 가을에 이루어집니다.

2. 탈건: 수확한 포도를 탈건하여 준비합니다. 이때 잘못된 포도나 잎, 줄기 등을 제거합니다.

3. 짜기: 준비된 포도를 짜서 주스를 추출합니다. 이 과정을 발효라고도 합니다.

4. 발효: 추출한 포도주스에 효모를 첨가하여 발효를 진행시킵니다. 발효 과정에서 설탕이 알코올로 변하며, 와인의 향과 맛이 형성됩니다.

5. 압착: 발효가 완료된 와인을 압착하여 과즙을 분리합니다.

6. 숙성: 압착한 와인을 숙성시켜 와인의 맛과 향이 더 풍부해지도록 합니다. 숙성은 다양한 방법으로 이루어질 수 있습니다.

7. 필터링 및 병입: 와인을 필터링하여 불순물을 제거하고, 병에 담아 보관합니다.

8. 보관: 와인은 적절한 온도와 습도에서 보관하여 최상의 맛과 향을 유지합니다.

이렇게 완성된 와인은 즐기는 사람들에게 즐거움과 만족을 주는 음료수가 됩니다.[llm/end] [1:llm:ChatOpenAI] [6.28s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "와인을 만드는 과정은 다음과 같습니다:\n\n1. 수확: 와인을 만들기 위한 포도를 수확합니다. 수확 시기는 포도의 성숙도와 품종에 따라 다르며, 보통 가을에 이루어집니다.\n\n2. 탈건: 수확한 포도를 탈건하여 준비합니다. 이때 잘못된 포도나 잎, 줄기 등을 제거합니다.\n\n3. 짜기: 준비된 포도를 짜서 주스를 추출합니다. 이 과정을 발효라고도 합니다.\n\n4. 발효: 추출한 포도주스에 효모를 첨가하여 발효를 진행시킵니다. 발효 과정에서 설탕

#5.0

In [6]:
# 메모리가 없으면 AI 모델은 이전의 대화를 기억하지 못한다. 그래서 우리는 memory를 사용해야한다.
#return_messages=True는 저장한 내용을 챗모델에게 사용 할 경우 True 아니면 False

#ConversationBufferMemory는 모든 대화 내용을 저장하는 방식
CB_memory = ConversationBufferMemory(return_messages=True)
# 모든 메모리는 save_context와 load_memory_variables를 갖고 있다. 각 메모리마다 작동 방식이 다를 뿐이다.
CB_memory.save_context({"input":"안녕!"},{"output":"안뇽!"})

CB_memory.load_memory_variables({})




{'history': [HumanMessage(content='안녕!'), AIMessage(content='안뇽!')]}

#5.1

In [11]:
#ConversationBufferWindowMemory는 대화의 특정 부분만을 저장하는 방식
CBW_memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4
)

def add_message(input,output):
    CBW_memory.save_context({"input":input},{"output":output})

add_message(1,1)
add_message(2,2)
add_message(3,3) # k가 4라서 이거 이전의 메세지들은 저장되지 않음
add_message(4,4)
add_message(5,5)
add_message(6,6)
add_message(7,7)

CBW_memory.load_memory_variables({})

{'history': [HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5'),
  HumanMessage(content='6'),
  AIMessage(content='6'),
  HumanMessage(content='7'),
  AIMessage(content='7')]}

#5.2

In [8]:
#ConversationSummaryMemory는 대화를 요약하여 저장하는 방식 / 초기에는 저장 공간이 많이 요구되지만, 이후에는 저장공간을 많이 사용되지 않음.
CS_memory = ConversationSummaryMemory(
    llm = CHAT
)

def add_message(input, output):
    CS_memory.save_context({"input": input}, {"output": output})

def get_history():
    return CS_memory.load_memory_variables({})

add_message("안녕 나는 유령개야. 오늘도 공부 중이야.", "와우! 완전 멋쪄")
get_history()

The human greets the AI in Korean and mentions that they are studying. The AI responds with enthusiasm.

{'history': 'The human greets the AI in Korean and mentions that they are studying. The AI responds with enthusiasm.'}

#5.3

In [12]:
#ConversationSummaryMemory는 특정 갯수의 메세지까지는 그냥 저장하고, 이후에 새로운 메세지에 의해 밀려나는 기존 메시지는 요약되어 저장됨
CSB_memory = ConversationSummaryBufferMemory(
    llm=CHAT,
    max_token_limit=100,
    return_messages=True,
)


def add_message(input, output):
    CSB_memory.save_context({"input": input}, {"output": output})


def get_history():
    return CSB_memory.load_memory_variables({})


add_message("안녕 나는 유령개야. 오늘도 공부 중이야.", "와우! 완전 멋쪄")
add_message("지금은 파이썬 공부중이야.", "와우! 파이썬 공부라니, 어떤걸 하고 있어?")
get_history()

The human greets the AI in Korean and mentions that they are studying today.

{'history': [SystemMessage(content='The human greets the AI in Korean and mentions that they are studying today.'),
  AIMessage(content='와우! 완전 멋쪄'),
  HumanMessage(content='지금은 파이썬 공부중이야.'),
  AIMessage(content='와우! 파이썬 공부라니, 어떤걸 하고 있어?')]}

#5.4

In [5]:
#ConversationKGMemory은 대화를 요약하면서 동시에 핵심적 키워드를 추출하는 방식
CKG_memory = ConversationKGMemory(
    llm=CHAT,
    return_messages=True,
)

def add_message(input, output):
    CKG_memory.save_context({"input": input}, {"output": output})


add_message("안녕 나는 유령개야. 오늘도 공부 중이야.", "와우! 완전 멋쪄")
add_message("지금은 파이썬 공부중이야.", "와우! 파이썬 공부라니, 어떤걸 하고 있어?")

NONENONE

In [6]:
CKG_memory.load_memory_variables({"inputs": "지금 무엇을 공부하나요"})

NONE

{'history': []}

#5.5

In [36]:
memory = ConversationSummaryBufferMemory(
    llm=CHAT,
    max_token_limit=80,
    return_messages=True, # 이것도 있어야 작동하는듯?
    memory_key="chat_history" # 하단 탬플릿의 chat_history에 메모리에 저장된 내역을 전달하기 위한 설정값
)

# 메모리에 대화는 저장되나, 해당 로그가 기록이 llm에 전달되지 않고있음. 그래서, 아래와 같은 탬플릿 생성
template = """
    당신은 인간과 대화할때 도움이 되는 인공지능입니다.

    {chat_history}
    Human: {question}
    You: 
"""

chain = LLMChain(
    llm=CHAT,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
#    verbose=True # 해당 값이 있으면 프롬프트가 실행될때 로그들을 확인 할 수 있음
)



chain.predict(question="내 이름은 유령개야!")

안녕, 유령개! 만나서 반가워요. 어떻게 도와드릴까요?

'안녕, 유령개! 만나서 반가워요. 어떻게 도와드릴까요?'

In [37]:
chain.predict(question="작명 사이트에서 이걸 추천해줬어!")

그럼 유령개라는 이름도 참 독특하고 멋있네요! 이름 추천 사이트에서 좋은 선택을 했네요. 어떤 도움이 필요하신가요?The human introduces themselves as "Ghost Dog." The AI greets Ghost Dog and asks how it can help. Ghost Dog mentions that a naming site recommended their name.

'그럼 유령개라는 이름도 참 독특하고 멋있네요! 이름 추천 사이트에서 좋은 선택을 했네요. 어떤 도움이 필요하신가요?'

In [38]:
chain.predict(question="내 이름이 뭔지 알아?")

네, 유령개님의 이름을 알고 있습니다. 어떤 도움이 필요하신가요?The human introduces themselves as "Ghost Dog" and the AI greets them, complimenting their unique name suggested by a naming site. The AI asks how it can help Ghost Dog.

'네, 유령개님의 이름을 알고 있습니다. 어떤 도움이 필요하신가요?'

In [24]:
memory.load_memory_variables({})

{'chat_history': [SystemMessage(content='The human introduces themselves as "Ghost Dog." The AI greets Ghost Dog and asks how it can help. The human mentions a naming website and the AI finds the name recommendation interesting and asks about the website.'),
  HumanMessage(content='내 이름이 뭔지 알아?'),
  AIMessage(content='당신의 이름은 "Ghost Dog"이라고 소개하셨죠. 저는 여러분의 대화 상대가 될 수 있습니다. 무엇을 도와드릴까요?')]}

#5.6

In [50]:
memory = ConversationSummaryBufferMemory(
    llm=CHAT,
    max_token_limit=120,
    return_messages=True,
    memory_key="chat_history" # 하단 탬플릿의 chat_history에 메모리에 저장된 내역을 전달하기 위한 설정값
)

prompt = ChatPromptTemplate.from_messages([
    ("system","당신은 인간과 대화할때 도움이 되는 인공지능입니다."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human","{question}"),
])

chain = LLMChain(
    llm=CHAT,
    memory=memory,
    prompt=prompt,
    verbose=True # 해당 값이 있으면 프롬프트가 실행될때 로그들을 확인 할 수 있음
)
chain.predict(question="내 이름은 유령개야!")



> Entering new LLMChain chain...
Prompt after formatting:
System: 당신은 인간과 대화할때 도움이 되는 인공지능입니다.
Human: 내 이름은 유령개야!
안녕, 유령개! 만나서 반가워. 무엇을 도와드릴까요?
> Finished chain.


'안녕, 유령개! 만나서 반가워. 무엇을 도와드릴까요?'

In [51]:
chain.predict(question="너랑은 나눌 이야기 없어!")



> Entering new LLMChain chain...
Prompt after formatting:
System: 당신은 인간과 대화할때 도움이 되는 인공지능입니다.
Human: 내 이름은 유령개야!
AI: 안녕, 유령개! 만나서 반가워. 무엇을 도와드릴까요?
Human: 너랑은 나눌 이야기 없어!
알겠어요, 만약 도움이 필요하거나 궁금한 점이 생기면 언제든지 물어주세요. 저는 여러분을 돕기 위해 여기 있어요.
> Finished chain.
The human introduces themselves as "유령개." The AI responds with a greeting and asks how it can help.

'알겠어요, 만약 도움이 필요하거나 궁금한 점이 생기면 언제든지 물어주세요. 저는 여러분을 돕기 위해 여기 있어요.'

In [52]:
chain.predict(question="난 게임을 좋아해. 특히 역사나 시뮬레이션. 아니면, 대전략류 게임을 좋아해")



> Entering new LLMChain chain...
Prompt after formatting:
System: 당신은 인간과 대화할때 도움이 되는 인공지능입니다.
System: The human introduces themselves as "유령개." The AI responds with a greeting and asks how it can help.
Human: 너랑은 나눌 이야기 없어!
AI: 알겠어요, 만약 도움이 필요하거나 궁금한 점이 생기면 언제든지 물어주세요. 저는 여러분을 돕기 위해 여기 있어요.
Human: 난 게임을 좋아해. 특히 역사나 시뮬레이션. 아니면, 대전략류 게임을 좋아해
게임을 좋아하시는군요! 역사나 시뮬레이션, 대전략 게임을 즐기시는 것 같아요. 그런 게임들은 정말로 흥미로운 선택이죠. 어떤 게임을 좋아하시는지 더 자세히 알려주시면, 해당 장르의 다양한 게임을 추천해드릴 수 있어요. 혹시 좋아하는 게임이나 특정 장르의 게임이 있나요?
> Finished chain.
The human introduces themselves as "유령개." The AI responds with a greeting and offers assistance. The human expresses their love for games, especially historical, simulation, or grand strategy games. The AI acknowledges their interest in those genres and offers to recommend games based on their preferences.

'게임을 좋아하시는군요! 역사나 시뮬레이션, 대전략 게임을 즐기시는 것 같아요. 그런 게임들은 정말로 흥미로운 선택이죠. 어떤 게임을 좋아하시는지 더 자세히 알려주시면, 해당 장르의 다양한 게임을 추천해드릴 수 있어요. 혹시 좋아하는 게임이나 특정 장르의 게임이 있나요?'

#5.7

In [67]:
memory = ConversationSummaryBufferMemory(
    llm=CHAT,
    max_token_limit=120,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","당신은 인간과 대화할때 도움이 되는 인공지능입니다."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | CHAT


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [62]:
invoke_chain("난 게임을 좋아해. 특히 역사나 시뮬레이션. 아니면, 대전략류 게임을 좋아해")

그렇군요! 게임을 좋아하는 건 멋진 취미네요. 역사나 시뮬레이션, 그리고 대전략류 게임을 즐기는 것 같아서 저도 흥미롭게 생각해요. 어떤 게임을 주로 즐기시나요? 함께 게임 이야기를 나눠보는 건 어떨까요?The human expresses their love for games, especially historical or simulation ones, and sometimes grand strategy games. The AI finds it fascinating and suggests sharing game stories together.content='그렇군요! 게임을 좋아하는 건 멋진 취미네요. 역사나 시뮬레이션, 그리고 대전략류 게임을 즐기는 것 같아서 저도 흥미롭게 생각해요. 어떤 게임을 주로 즐기시나요? 함께 게임 이야기를 나눠보는 건 어떨까요?'


In [63]:
invoke_chain("최근에는 문명6 많이 한 거 같아")

문명 6는 정말 인기 있는 게임이죠! 어떤 문명을 선택해서 플레이하는지 궁금하네요. 어떤 전략을 사용하거나 특별한 이야기가 있나요? 함께 게임 이야기를 공유해보는 건 어떨까요?The human expresses their love for games, especially historical or simulation ones, and sometimes grand strategy games. The AI finds it fascinating and suggests sharing game stories together. The human mentions playing Civilization 6 a lot recently.content='문명 6는 정말 인기 있는 게임이죠! 어떤 문명을 선택해서 플레이하는지 궁금하네요. 어떤 전략을 사용하거나 특별한 이야기가 있나요? 함께 게임 이야기를 공유해보는 건 어떨까요?'


In [64]:
invoke_chain("내가 어떤 게임을 자주했다고 말했는지 기억해?")

네, 당신이 최근에 Civilization 6를 자주 플레이한다고 말씀하셨죠! 역사적이고 시뮬레이션 요소가 있는 게임을 좋아하신다고 했던 것 같아요. 어떤 문명을 선택하고 어떤 전략을 사용하는지 궁금합니다! 함께 게임 이야기를 나누면 재미있을 것 같네요.The human expresses their love for games, especially historical or simulation ones, and sometimes grand strategy games. The AI finds it fascinating and suggests sharing game stories together. The human mentions playing Civilization 6 a lot recently. The AI remembers the human's preference for Civilization 6 and their interest in historical and simulation games. They suggest sharing game stories and are curious about the civilizations chosen and strategies used.content='네, 당신이 최근에 Civilization 6를 자주 플레이한다고 말씀하셨죠! 역사적이고 시뮬레이션 요소가 있는 게임을 좋아하신다고 했던 것 같아요. 어떤 문명을 선택하고 어떤 전략을 사용하는지 궁금합니다! 함께 게임 이야기를 나누면 재미있을 것 같네요.'
